In [ ]:
import os
import sys
# os.chdir('/Users/ludo/Desktop/animalai/animalai')
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")
from animalai.envs.gym.environment import AnimalAIGym
import random
from animalai.envs.arena_config import ArenaConfig

import yaml
import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# CONFIG
env_path = '../../env/aaiagain'
worker_id = 3
#random.randint(1, 100)
# competition_envs = os.listdir('../competition_configurations/')

seed = 1


# yaml_path = '../examples/rand_env.yml'
# ac = ArenaConfig(yaml_path)
# arena_path = "../../competition_configurations/2-3-1.yml" #'configurations/curriculum/0.yaml'
# arena_path = '../../competition_configurations/9-5-1.yml'
arena_path = '../training_set/ymaze2.yml'
# arena_path = '../configurations/ramps/0.yml'

# ac = ArenaConfig(arena_path)
# ac.arenas[1] = copy.deepcopy(ac.arenas[0])
# yaml_path = 'configurations/curriculum4/19.yml'
ac = ArenaConfig(arena_path)
env = AnimalAIGym(environment_filename=env_path,
              worker_id=worker_id,
              n_arenas=1,
              arenas_configurations=ac,
                use_visual=False,
                seed=seed,
                 grayscale=False,
                 resolution=84)

x = env.step([[0,0]])

In [ ]:
import os
os.chdir('..')

In [ ]:
from animalai.envs.cvis import ExtractFeatures
# from utils import goal_on_platform as gop
# arena_path = "../../competition_configurations/2-3-1.yml" #'configurations/curriculum/0.yaml'
# ac = ArenaConfig(arena_path)
env._env.train =False
env.reset(ac)

plt.imshow(env.render())

ef = ExtractFeatures(display=False, training=False)
# for i in range(20):

state = ef.run(env.render())
#     goals = state['goal1']
#     if len(goals)>1:
#         count = [i for i in goals if i[1]]
#         left = [i for i in count if i[0][0]<0.5]
#         right = [i for i in count if i[0][0]>0.5]
#         direction = "left" if len(left)>len(right) else 'right'
# #         print( f"more_goals({direction}).\n")
#         print(direction)
def goal_on_platform(state):
    img = state['visual_obs']
    dim = img.shape[0]
    state = ef.run(img)
    if not state['goal']:
        return None
    goal = state['goal'][0][0]
    under_goal = [goal[0], goal[1]+goal[3], goal[2], goal[3]]
    selector = [dim*(under_goal[0]), dim*(under_goal[1]), dim*under_goal[2], dim*under_goal[3]]
    selector = [int(np.ceil(i)) for i in selector]
    res = img[selector[1]:selector[1]+selector[3],selector[0]:selector[0]+selector[2],:]
    res = img[1:1,1:2,:]
    plt.imshow(res)
    res = ef.run(res)
    print(res)
    if bool(res['platform']):
        return True
    return False
goal_on_platform({'visual_obs':env.render()})


In [ ]:
goal = state['goal'][0][0]
# under_goal=goal
under_goal = [goal[0], goal[1]+goal[3], goal[2], goal[3]]
print(under_goal)
# x = env.render()
selector = [84*(under_goal[0]), 84*(under_goal[1]), 84*under_goal[2], 84*under_goal[3]]
selector = [int(np.ceil(i)) for i in selector]
print(selector)
res= x[selector[1]:selector[1]+selector[3],selector[0]:selector[0]+selector[2],:]
res = x[50:51, 40:41,:]
plt.imshow(res)

ef.run(res)

In [ ]:
# If top-left inner box corner is inside the bounding box
def box_in(bb1, bb2):
    innerb = {'x':bb1[0], 'y':bb1[1],'width':bb1[2], 'heigher':bb1[3]}
    boundb = {'x':bb2[0], 'y':bb2[1],'width':bb2[2], 'heigher':bb2[3]}
    if boundb['x'] < innerb['x'] and boundb['y'] < innerb['y']:
        print('yep')
        # If bottom-right inner box corner is inside the bounding box
        if innerb['x'] + innerb['width'] < boundb['x'] + boundb['width'] \
                and innerb['y'] + innerb['height'] < boundb['y'] + boundb['height']:
            print('The entire box is inside the bounding box.')
        else:
            print('Some part of the box is outside the bounding box.')
    else:
        print('na')
box_in(under_goal, state['platform'][0][0])

In [ ]:
# yaml_path = f"../../competition_configurations/8-19-2.yml"
yaml_path = '../training_set/numerosity.yml'

ac = ArenaConfig(yaml_path)
env.reset(ac)
plt.imshow(env.render())
def convert_dimensions(func):
    def wrapper(*dimensions):
        """Convert from x,y,w,h to x1, x2, y1, y2"""
        # x is top left corner
        res = []
        for dims in dimensions:
            x1, y1 = dims[0], dims[1] #x, y
            x2 = x1 + dims[2] #w
            y2 = y1 + dims[3] #h
            res.append({'x1': x1, 'y1':y1, 'x2':x2, 'y2':y2})


        return func(*res)
    return wrapper
@convert_dimensions
def get_distance(dims1, dims2):
    """Get shortest distance between two rectangles"""

    x1, y1, x1b, y1b = dims1.values()
    x2, y2, x2b, y2b = dims2.values()
    left = x2b < x1
    right = x1b < x2
    bottom = y2b < y1
    top = y1b < y2
    dist = lambda x,y: np.linalg.norm(np.array(x)-np.array(y))
    if top and left:
        return dist((x1, y1b), (x2b, y2))
    elif left and bottom:
        return dist((x1, y1), (x2b, y2b))
    elif bottom and right:
        return dist((x1b, y1), (x2, y2b))
    elif right and top:
        return dist((x1b, y1b), (x2, y2))
    elif left:
        return x1 - x2b
    elif right:
        return x2 - x1b
    elif bottom:
        return y1 - y2b
    elif top:
        return y2 - y1b
    else:             # rectangles intersect
        return 0.
@convert_dimensions
def get_overlap(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    # assert bb1['x1'] < bb1['x2']
    # assert bb1['y1'] < bb1['y2']
    # assert bb2['x1'] < bb2['x2']
    # assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
from animalai.envs.cvis import ExtractFeatures
ef = ExtractFeatures(display=False, training=False)
state = ef.run(env.render())
goals = [state['goal'][0][0]]#[i[0] for i in state['obj'] if i[1]=='goal']
lava = [i[0] for i in state['lava']]#[i[0] for i in state['obj'] if i[1]=='lava']
# goals = [[i[0], i[1], i[w]]] 
goals = [[i[0]-0.1,i[1], i[2]+0.2, 1-i[1]] for i in goals]
for i in goals:
    for j in lava:
#         print(i,j)
        print(get_overlap(i,j))

In [ ]:
from animalai.envs.cvis import ExtractFeatures
ef = ExtractFeatures(display=False, training=False)
def first_steps(env):
    moving = False
    move_count = 0
    obj = ef.run(env.render())
    for i in range(5):
        res = env.step([0,0])
        obj_1 = ef.run(env.render())
        for v,v1 in zip(obj.values(), obj_1.values()):
            if v:
                o = sum(v[0][0])#* v[0][0][1]
                o1 = sum(v1[0][0])#* v1[0][0][1]
                if 'goal'==v1[0][1]:
                    print(v1[0])
                if o1!=o:
                    move_count+=1
        obj = obj_1
        if move_count>1:
            moving = True
            break
    print("MOV", move_count)
    return res, moving
first_steps(env)

In [ ]:
from animalai.envs.cvis import ExtractFeatures
ef = ExtractFeatures(display=False, training=False)
env.reset()
obj = ef.run(env.render())
move_count = 0
# for i in range(10):
#     env.step([2,0])

plt.imshow(env.render())
# img = env.render()
# img = (img*255)[:,:,::-1].astype(np.uint8)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# # plt.imshow(ef.img)
# import cv2
# cv2.countNonZero(img)

In [ ]:
# for i in range(10):
#     x = env.step([0,0])
bool(np.max(x[0]))
# plt.imshow(env.render())


In [ ]:
ac.arenas[0].t

In [ ]:
os.chdir('..')
import config as cfg
ramps = sorted(cfg.COMPETITION_CONFIGS['Ramp Usage'])

for i in ramps:
    yaml_path = f"../competition_configurations/{i}.yml"
    print(i)
    ac = ArenaConfig(yaml_path)
    env.reset(ac)
    plt.imshow(env.render())
    plt.show()
# plt.savefig('rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)

# env._env.arenas_obj_config_side_channel.arenas_config

In [ ]:
import time
for i in range(10):
    x = env.step([[1,1]])
    print(i, x[3]['batched_step_result'].obs[1][0][1])
plt.imshow(env.render())
# plt.savefig("/Users/ludo/Desktop/bam.png",bbox_inches='tight',transparent=True, pad_inches=0)

In [ ]:
import os
os.chdir('..')
import sys
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")

In [ ]:
from animalai.envs.cvis import ExtractFeatures
from matplotlib import pyplot as plt
import cv2
ef = ExtractFeatures(display=True, training=False)

for i in os.listdir('./fake_test'):
    img = cv2.imread(f'./fake_test/{i}')[60:415, 145:505,:]
#     plt.imshow(img[:,:,::-1])
#     plt.show()
#     print(i)
    bb = ef.run(img[:,:,::-1]/255)
    if(any(len(bb[i])>1 for i in bb)):
#         msk = ef.run(img[:,:,::-1]/255, mode='mask',mask='ramp')
#     print(len(ef.run(img[:,:,::-1]/255)['ramp']), i)
        print(i)
        print(bb['wall'])

        plt.imshow(ef.img[:,:,::-1])
        plt.show()
#         plt.imshow(msk[:,:])
#         plt.show()

In [ ]:
bb

In [ ]:
import cv2
# for i in range(10):
#     env.step([[0,2]])
plt.imshow(env.render())
img = env.render().copy()
img = (img*255)[:,:,::-1].astype(np.uint8)
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
hsv = hsv_cls.blue
mask = cv2.inRange(hsv_img, hsv[0], hsv[1])
res = cv2.bitwise_and(hsv_img, hsv_img, mask=mask)[:,:,2]
# res[res > 0] = 255
res = res/255
cv2.imwrite("/Users/ludo/Desktop/bam.png", res)

In [ ]:
:~ initiate(rotate).[-1@1]
:~ initiate(balance(V2,V3)), on(agent,V1).[-1@3, V1, V2, V3]
:~ initiate(climb(V1)), ramp(V1).[-1@2, V1]

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%% Pre-processing                          : 0.001s
%% Hypothesis Space Generation             : 1.865s
%% Conflict analysis                       : 17.262s
%%   - Brave Orders                        : 17.262s
%% Counterexample search                   : 1.747s
%%   - CDOEs                               : 0.891s
%%   - CDPIs                               : 0.855s
%% Hypothesis Search                       : 90.796s
%% Propagation                             : 27.035s
%%   - CDOEs                               : 27.035s
%% Total                                   : 142.657s
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [ ]:
class HSV:
	"""Color lower and upper bounds in HSV format"""

	@classmethod
	def __getattribute__(cls,attr):
		return [np.array(i) for i in getattr(cls, attr)]

	green = [[33,80,40], [102,255,255]]
	red = [[0,162,142], [179,203,188]]
	grey = [[0,0,0], [0,0,224]]
	orange = [[20,121,158], [23,255,255]]
	brown = [[7,53,40], [18,87,121]]
	blue = [[119, 255, 106], [120, 255, 255]]


hsv_cls = HSV()
# objects['platform'] = hsv_cls.blue

# objects = {
# 	'goal': hsv_cls.green,
# 	# 'danger_zone': hsv_cls.red,
# 	# 'wall': hsv_cls.grey,
# }
# ef.get_contour(objects['goal'])
# plt.imshow(ef.hsv_img)

In [ ]:
from copy import deepcopy
ac = deepcopy(ArenaConfig(arena_path))
ac.arenas = {k:v for k,v in ac.arenas.items() if k==1}
ac.arenas

In [ ]:
env._env.rl_output.agentInfos["AnimalAI?team=0"].value[0].reward

In [ ]:
os.chdir('..')
import time
from config import COMPETITION_CONFIGS as comp
for i in comp['Numerosity']:
    print(i)
    yaml_path = f'../competition_configurations/{i}.yml'
    ac = ArenaConfig(yaml_path)    
    env.reset(ac)
    for i in range(30):
        env.step([0,0])
    time.sleep(1)

In [ ]:
env.close()

In [ ]:
x[3]['batched_step_result'].reward


In [ ]:
from animalai.envs.cvis_img import ExtractFeatures
ef = ExtractFeatures(display=True, training=False)
y = ef.run_mask(env.render())
plt.imshow(ef.img[:,:,::-1])
plt.savefig('/Users/ludo/Desktop/rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)


In [ ]:
from animalai.envs.cvis import ExtractFeatures
ef = ExtractFeatures(display=True, training=False)
data = x[3]['batched_step_result'].obs[0][0]
print(ef.run(data))
plt.imshow(ef.img)

In [ ]:
class HSV:
	"""Color lower and upper bounds in HSV format"""

	@classmethod
	def __getattribute__(cls,attr):
		return [np.array(i) for i in getattr(cls, attr)]

	green = [[33,80,40], [102,255,255]]
	# red = [[0,181,0], [6,255,255]]
	# red = [[0,184,0], [179,208,217]]
	red = [[125,16,88], [179,255,255]]
	grey = [[0,0,0], [0,0,224]]
	brown = [[7,53,40], [18,87,121]]

hsv_cls = HSV()
objects = {
	'goal': hsv_cls.green,
	# 'danger_zone': hsv_cls.red,
	# 'wall': hsv_cls.grey,
}
ef.get_contour(objects['goal'])
plt.imshow(ef.hsv_img)

In [ ]:
# env._env.DEFAULT_EDITOR_PORT
x[3]['batched_step_result'].obs[1].reshape(-1,9)[[0,11,22]][:, 0:6].flatten()

In [ ]:
env.close()

In [ ]:
env.reset()
for _ in range(1):
    x = env.step([[0,0]])
    res = x[3]['batched_step_result'].obs[1][0]
    plt.axis('off')
    plt.imshow(res)
#     plt.savefig('rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)
    plt.show()
    res = x[3]['batched_step_result'].obs[1][1]
    plt.axis('off')
    plt.imshow(res)
#     plt.savefig('rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)
    plt.show()    

In [ ]:
import sys
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")
from animalai.envs.gym.environment import AnimalAIGym
import random
from animalai.envs.arena_config import ArenaConfig

import os
import yaml
import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from animalai.envs.environment import AnimalAIEnvironment

# CONFIG
env_path = '../../env/AnimalAI'
# env_path = '../env/AnimalAI'

worker_id = 2
seed = 1
# yaml_path = '../configurations/ramps/0.yml'

# yaml_path = '../competition_configurations/3-20-1.yml'
# yaml_path = '../configurations/curriculum12/112.yml'
# yaml_path = "../../competition_configurations/5-8-3.yml" #'configurations/curriculum/0.yaml'
# yaml_path = "slack.yml"
# yaml_path = '../training_set/ramp2.yml'
yaml_path = 'test.yml'
# yaml_path = f"../../competition_configurations/4-5-1.yml"

ac = ArenaConfig(yaml_path)

env = AnimalAIEnvironment(
                    file_name=env_path,
                    arenas_configurations = ac,
                    worker_id=worker_id,
                    play=True, base_port=5002,
                    alter_obs = False,
                    train=False)
# env._env.train =False
# env = AnimalAIGym(environment_filename=env_path,
#               worker_id=worker_id,
#               n_arenas=1,
#               arenas_configurations=ac,

#                 seed=seed,
#                  grayscale=False)
x = env.step()

# plt.axis('off')
# plt.imshow(env.visual_obs)
# plt.savefig('rand_env.png',bbox_inches='tight',transparent=True, pad_inches=0)


In [ ]:
ac.arenas[0].pass_mark


In [ ]:
env.get_step_result('AnimalAI?team=0').done


In [ ]:
0 is ascending towards bottom
90 is ascending towards left
180 ascending towards top
270 ascending towards right

